# # INTRODUCCIÓN

# Optimización de Arquitecturas de Deep Learning para Detección y Conteo Automático de Fauna en Surveys Aéreos de Alta Resolución

## Proyecto Guacamaya: Sistema Automatico de Detección de Fauna Africana

**Instituciones:** Microsoft AI for Good Lab, Centro SINFONÍA - Universidad de los Andes, Instituto Sinchi, Instituto Alexander von Humboldt

**Grupo 12:** Jorge Mario Guaquetá, Daniel Santiago Trujillo, Inmaculada Concepción Rondón, Daniela Alexandra Ortiz Santacruz

**Dataset:** DelPlan 2022 - ~2,000 imágenes aéreas de 5000x4000 píxeles (20MP)
**Especies:** Buffalo, Elephant, Kudu, Topi, Warthog, Waterbuck

# 1. DESCRIPCIÓN DEL PROYECTO

## 1.1 Contexto

Los *Aerial Wildlife Surveys* son fundamentales para el monitoreo poblacional de fauna en ecosistemas extensos. El método tradicional (conteo manual) presenta limitaciones críticas: fatiga visual, errores por turbulencia, tiempo limitado de observación, variabilidad inter-observador y costos elevados.

## 1.2 Problema Principal

Las imágenes aéreas de ultra-alta resolución (5000×4000 píxeles) exceden las capacidades de memoria de GPUs estándar. La arquitectura HerNet actual presenta:

- **Restricciones de memoria:** imposibilidad de procesar imágenes completas
- **Detecciones duplicadas:** animales grandes generan múltiples puntos cercanos  
- **Post-procesamiento ineficiente:** falta de consolidación de detecciones

**Arquitectura Híbrida Multi-Scale (AHMS)** con tres componentes:

- **Patch Management Inteligente (PMI):** Segmentación adaptativa con *overlapping*
- **Multi-Scale Feature Fusion (MSFF):** Procesamiento en múltiples resoluciones  
- **Post-Processing Optimization (PPO):** Clustering espacial y NMS adaptativo


## 1.4 Impacto Esperado

- **Automatización completa** de *surveys* eliminando errores humanos
- **Reducción de costos** operativos
- **Escalabilidad** para monitoreo continuo  
- **Contribución a repositorio open-source** (+65,000 descargas)

# 2. OBJETIVOS DEL PROYECTO

## 2.1 Objetivo General

Desarrollar y validar una arquitectura optimizada de *deep learning* para detección y conteo automático de fauna en imágenes aéreas de ultra-alta resolución, superando las limitaciones actuales de HerNet.

## 2.2 Objetivos Específicos

1. Optimizar manejo de memoria con estrategias eficientes de *patchado*
2. Implementar algoritmos avanzados de post-procesamiento
3. Comparar arquitecturas (HerNet vs YOLO vs híbridas)
4. Desarrollar técnicas de muestreo inteligente para balance de datos
5. Validar robustez bajo condiciones variables
6. Establecer métricas especializadas para *wildlife counting*

## 2.3 Metas de Mejora (vs HerNet Original)

| Métrica | Estado Actual | Meta | Mejora |
|---------|---------------|------|---------|
| **AP@0.5** | 0.67 | >0.75 | +12% |
| **MAE** | 4.2 | <3.0 | -30% error |
| **MAPE** | 18.3% | <12% | -35% error |
| **Tiempo de Procesamiento** | ~45s | <20s | Optimización |


# 3. PREGUNTAS DE INVESTIGACIÓN

## 3.1 Preguntas Técnicas

1. **Estrategia de patchado:** ¿Cuál es la estrategia óptima de *patchado* (tamaño, *overlapping*) para imágenes de 20MP que maximice detecciones y minimice uso de memoria?

2. **Arquitectura óptima:** ¿Qué arquitectura (HerNet, YOLOv8, YOLOv11, DETR, o híbrida) ofrece el mejor balance entre precisión, *recall* y velocidad para *point detection*?

3. **Detecciones duplicadas:** ¿Cómo eliminar eficientemente detecciones duplicadas sin perder animales reales en áreas de alta densidad?

4. **Data augmentation:** ¿Qué técnicas de *data augmentation* son más efectivas para simular condiciones reales de iluminación, clima y terreno?

5. **Balance de datos:** ¿Cuál es el *ratio* óptimo de *patches* positivos/negativos durante entrenamiento para evitar sesgo hacia *backgrounds*?

## 3.2 Preguntas de Aplicación

1. **Generalización a nuevas especies:** ¿Cómo generaliza el modelo a nuevas especies no vistas en entrenamiento?

2. **Métricas para conservacionistas:** ¿Qué métricas reflejan mejor las necesidades reales de conservacionistas (error de conteo vs precisión de localización)?

3. **Robustez ambiental:** ¿Cómo afectan variaciones estacionales y de *habitat* la performance del modelo?

4. **Estimación de densidad:** ¿Es posible estimar densidad poblacional además del conteo individual?

5. **Transferibilidad:** ¿Cómo transferir el modelo a otros ecosistemas (Amazonía, Serengeti, etc.)?

# 4. METODOLOGÍA

## 4.1 Entendimiento y Preparación de los Datos
### Configuración del Entorno
El procesamiento se realizó en **Google Colab Pro** para aprovechar mayor potencia de GPU, montando Google Drive para acceder al dataset de ~2,000 imágenes de 20MP.

### Estructura  &  Distribución del Dataset
- **Train:** 928 imágenes
- **Validation:** 111 imágenes  
- **Test:** 258 imágenes

### Herramientas y Frameworks
Para el desarrollo se utilizó Ultralytics YOLO para la implementación de modelos de detección de objetos.

In [ ]:
!pip install ultralytics

### 4.2 Entrenamiento del Modelo Base

Se implementó YOLO11s como arquitectura base para la detección de fauna, configurando un entrenamiento inicial de 20 épocas para validación rápida del pipeline.

```python
from ultralytics import YOLO

# Configuración inicial del modelo YOLO11s
model = YOLO("yolo11s.pt")

# Parámetros de entrenamiento rápido para validación
results = model.train(
    data=DATA_YAML,
    epochs=20,
    imgsz=1024,
    batch=8,
    project=RESULTS_DIR,
    name="retrain_yolo11s"
)

### 4.3 Resultados del Entrenamiento Inicial

El entrenamiento de 20 épocas completó en **1.12 horas**, obteniendo los siguientes resultados de validación:

**Métricas Principales:**
- **mAP@50:** 0.332
- **Precisión (Box P):** 0.317  
- **Recall (R):** 0.365
- **mAP@50-95:** 0.11

**Rendimiento por Especie:**
- **Kudu (K):** mAP@50 = 0.556 (mejor rendimiento)
- **Buffalo (B):** mAP@50 = 0.495
- **Elephant (E):** mAP@50 = 0.400
- **Warthog (WH):** mAP@50 = 0.0515 (mayor desafío)

**Inferencia:** 10.0ms por imagen (Tesla T4)

4.4.4 - Configuración y Carga de Datos

In [ ]:
# ANÁLISIS DE RESULTADOS DEL ENTRENAMIENTO YOLO
import pandas as pd
import matplotlib.pyplot as plt

# Configuración para gráficos profesionales
plt.style.use('default')

# Cargar resultados del entrenamiento
csv_path = "/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11/retrain_yolo11s/results.csv"
df = pd.read_csv(csv_path)

print(f"Total de epochs entrenadas: {len(df)}")
print(f"Métricas disponibles: {list(df.columns)}")

 4.4.5 - Análisis de Pérdidas (Loss)

In [ ]:
# 1. GRÁFICO DE PÉRDIDAS - EVOLUCIÓN DEL ENTRENAMIENTO
plt.figure(figsize=(12, 6))
plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss", linewidth=2)
plt.plot(df["epoch"], df["val/box_loss"], label="Val Box Loss", linewidth=2)
plt.plot(df["epoch"], df["train/cls_loss"], label="Train CLS Loss", linewidth=2)
plt.plot(df["epoch"], df["val/cls_loss"], label="Val CLS Loss", linewidth=2)
plt.title("Evolución de Pérdidas (Loss) - Entrenamiento YOLO", fontsize=14, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Análisis de convergencia
final_train_loss = df["train/box_loss"].iloc[-1]
final_val_loss = df["val/box_loss"].iloc[-1]
print(f"Pérdida final - Train: {final_train_loss:.3f}, Val: {final_val_loss:.3f}")

4.4.6 - Métricas de Precisión y Recall

In [ ]:
# 2. PRECISIÓN Y RECALL - MÉTRICAS DE CALIDAD
plt.figure(figsize=(12, 6))
plt.plot(df["epoch"], df["metrics/precision(B)"], label="Precision", linewidth=2, color='blue')
plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall", linewidth=2, color='red')
plt.title("Precisión y Recall por Epoch", fontsize=14, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)
plt.show()

# Valores finales
final_precision = df["metrics/precision(B)"].iloc[-1]
final_recall = df["metrics/recall(B)"].iloc[-1]
print(f"Métricas finales - Precisión: {final_precision:.3f}, Recall: {final_recall:.3f}")


4.4.7 - Métricas mAP (Desempeño Principal)

In [ ]:
# 3. mAP - MÉTRICA PRINCIPAL DE DESEMPEÑO
plt.figure(figsize=(12, 6))
plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP50", linewidth=2, color='green')
plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP50-95", linewidth=2, color='orange')
plt.title("mAP50 y mAP50-95 - Desempeño de Detección", fontsize=14, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)
plt.show()

# Valores finales mAP
final_map50 = df["metrics/mAP50(B)"].iloc[-1]
final_map50_95 = df["metrics/mAP50-95(B)"].iloc[-1]
print(f"mAP final - mAP50: {final_map50:.3f}, mAP50-95: {final_map50_95:.3f}")


## 4.4.8 Análisis e Interpretación de Resultados Iniciales

Los resultados iniciales obtenidos tras 20 epochs de entrenamiento reflejan un comportamiento esperado dado el contexto experimental. El modelo evidencia capacidades incipientes de aprendizaje, particularmente en las clases mayoritarias (species_B, species_K y species_E), donde se observan señales claras de convergencia. Sin embargo, las clases minoritarias (species_WH y species_WB) presentan un rendimiento limitado, atribuible directamente a su baja representación en el conjunto de datos. El valor de mAP50 global de 0.33 se sitúa dentro de los rangos esperados para esta fase temprana de entrenamiento, confirmando que el modelo se encuentra en etapa de 'calentamiento' (warm-up). Estos resultados constituyen una base sólida que, mediante estrategias de aumento de epochs, balanceo de clases y técnicas de data augmentation, permitirá alcanzar mejoras significativas en el desempeño del modelo.

## 5.1 Análisis de Estructura del Dataset HerdNet

**PROBLEMA CRÍTICO DETECTADO:** Al examinar la estructura del dataset HerdNet, se identificó que los splits (train, val, test) contienen únicamente imágenes sin los archivos de anotaciones correspondientes en formato YOLO (.txt con bounding boxes).

**IMPLICACIÓN METODOLÓGICA:** Esta limitación requiere la creación manual de las anotaciones y conversión al formato YOLO antes de poder proceder con el entrenamiento de los modelos de detección.

**IMPACTO EN EL FLUJO:** Este hallazgo justifica la necesidad de desarrollar un pipeline de preprocesamiento adicional para la conversión de formatos.

## 5.2 Análisis de Estructura de Anotaciones HerdNet

Una vez ubicadas las anotaciones, se identificó que están organizadas en dos formatos:
- **groundtruth/json/** (anotaciones en JSON)
- **groundtruth/csv/** (anotaciones en CSV)

La inspección de los archivos CSV revela la estructura de anotaciones necesaria para la conversión a formato YOLO.

##5.3 - Inspección de Estructura CSV y Metadatos

In [ ]:
# 5.3 Inspección de Estructura CSV y Metadatos
import pandas as pd
from pathlib import Path

BASE_CSV = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/HerdNet_raw/extracted/general_dataset/groundtruth/csv")

# Cargar datos de anotaciones
train_csv = BASE_CSV / "train_big_size_A_B_E_K_WH_WB.csv"
df_train = pd.read_csv(train_csv)

print("ESTRUCTURA DE ANOTACIONES HERDNET:")
print("Columnas:", list(df_train.columns))
print("\nTotal de anotaciones por split:")
print(f"Train: {len(df_train)} anotaciones")
print(f"Val: {len(pd.read_csv(BASE_CSV / 'val_big_size_A_B_E_K_WH_WB.csv'))} anotaciones")
print(f"Test: {len(pd.read_csv(BASE_CSV / 'test_big_size_A_B_E_K_WH_WB.csv'))} anotaciones")

print("\nMUESTRA DE ANOTACIONES (primeras 5 filas):")
display(df_train.head())

## 5.4 Requisitos para Conversión a Formato YOLO

El análisis de las anotaciones HerdNet revela que utilizan formato de bounding boxes absoluto (x1, y1, x2, y2), mientras que YOLO requiere coordenadas normalizadas (clase, xc, yc, w, h) en el rango [0, 1].

**Problema crítico:** Para realizar esta conversión, es esencial conocer las dimensiones exactas (ancho y alto) de cada imagen, ya que HerdNet utiliza imágenes de ultra-alta resolución (4000×3000 o 6000×4000 píxeles).

**Implicación:** Sin esta información dimensional, el modelo YOLO entrenaría sin referencia espacial precisa, comprometiendo severamente la precisión de las detecciones.

##5.5 - Pipeline de Conversión a Formato YOLO

**Objetivos del proceso de conversión:**
- Procesar automáticamente los splits train/val/test
- Localizar las imágenes correspondientes para cada split
- Convertir coordenadas absolutas (x1, y1, x2, y2) → YOLO normalizado (xc, yc, w, h)
- Crear estructura de carpetas YOLO estándar para entrenamiento
- Generar archivos .txt con anotaciones en formato YOLO

**Dataset final listo para entrenamiento de YOLOv8/YOLO11**

##5.6 - Implementación de Conversión YOLO

In [ ]:
##5.6 Implementación de Conversión HerdNet a YOLO

import pandas as pd
from pathlib import Path
from PIL import Image
import os

# Configuración de rutas y dimensiones
ROOT = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/HerdNet_raw/extracted/general_dataset")
OUTPUT = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet")
OUTPUT.mkdir(parents=True, exist_ok=True)

W, H = 6000, 4000  # Dimensiones estándar HerdNet
splits = ["train", "val", "test"]

def voc_to_yolo(x1, y1, x2, y2, img_w=W, img_h=H):
    """Convierte coordenadas VOC absolutas a YOLO normalizadas"""
    xc = (x1 + x2) / 2 / img_w
    yc = (y1 + y2) / 2 / img_h
    w  = (x2 - x1) / img_w
    h  = (y2 - y1) / img_h
    return xc, yc, w, h

# Procesamiento por split
print("INICIANDO CONVERSIÓN HERDNET → YOLO")
print("=" * 40)

for split in splits:
    print(f"\nProcesando split: {split.upper()}")

    csv_path = ROOT / "groundtruth" / "csv" / f"{split}_big_size_A_B_E_K_WH_WB.csv"
    img_dir = ROOT / split
    out_img = OUTPUT / split / "images"
    out_lbl = OUTPUT / split / "labels"

    out_img.mkdir(parents=True, exist_ok=True)
    out_lbl.mkdir(parents=True, exist_ok=True)

    # Cargar y procesar anotaciones
    df = pd.read_csv(csv_path)
    df = df.rename(columns={"Image": "image"})
    print(f"  → Anotaciones cargadas: {len(df)}")

    for img_name, group in df.groupby("image"):
        src = img_dir / img_name
        dst_img = out_img / img_name

        # Copiar imagen si no existe
        if src.exists() and not dst_img.exists():
            os.system(f"cp '{src}' '{dst_img}'")

        # Escribir anotaciones YOLO
        lbl_path = out_lbl / (img_name.replace(".JPG", ".txt").replace(".jpg", ".txt"))
        with open(lbl_path, "w") as f:
            for _, row in group.iterrows():
                xc, yc, w, h = voc_to_yolo(row["x1"], row["y1"], row["x2"], row["y2"])
                cls = int(row["Label"])
                f.write(f"{cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")

    print(f"   Split {split} convertido correctamente")

print(f"\n" + "=" * 40)
print("DATASET YOLO GENERADO EXITOSAMENTE")
print(f"Ubicación final: {OUTPUT}")


## 5.7 Configuración del Archivo data.yaml para YOLO

Para completar la preparación del dataset YOLO, se genera el archivo de configuración `data.yaml` que define la estructura del dataset y el mapeo de clases para el entrenamiento.

##5.8 - Implementación del data.yaml

In [ ]:
# 5.8 Implementación del Archivo de Configuración data.yaml

from pathlib import Path
import yaml

DATASET_ROOT = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet")
yaml_path = DATASET_ROOT / "data.yaml"

# Configuración del dataset YOLO
data = {
    "path": str(DATASET_ROOT),
    "train": "train/images",
    "val": "val/images",
    "test": "test/images",
    "nc": 6,  # Número de clases en HerdNet
    "names": [
        "class_0",  # Especie A
        "class_1",  # Especie B
        "class_2",  # Especie E
        "class_3",  # Especie K
        "class_4",  # Especie WH
        "class_5"   # Especie WB
    ]
}

# Guardar archivo de configuración
with open(yaml_path, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print(" ARCHIVO data.yaml CONFIGURADO")
print(f"Ubicación: {yaml_path}")
print(f"Clases configuradas: {data['nc']}")
print(f"Estructura de paths: {data['train']}, {data['val']}, {data['test']}")

##5.9 - Validación de Integridad del Dataset YOLO

In [ ]:
# 5.9 Validación de Integridad del Dataset YOLO

import os
from pathlib import Path

dataset_root = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet")

def validar_pares_imagenes_labels(split):
    """Valida que cada imagen tenga su archivo de labels correspondiente"""
    path_imagenes = dataset_root / split / "images"
    path_labels = dataset_root / split / "labels"

    imagenes = os.listdir(path_imagenes)
    labels = os.listdir(path_labels)

    # Normalizar nombres (sin extensión)
    nombres_imagenes = {img.replace(".jpg", "").replace(".JPG", "") for img in imagenes}
    nombres_labels = {lbl.replace(".txt", "") for lbl in labels}

    # Encontrar discrepancias
    labels_faltantes = nombres_imagenes - nombres_labels
    imagenes_faltantes = nombres_labels - nombres_imagenes

    print(f"\n VALIDACIÓN {split.upper()}:")
    print(f"   Imágenes: {len(imagenes)} | Labels: {len(labels)}")

    if not labels_faltantes and not imagenes_faltantes:
        print("    INTEGRIDAD CONFIRMADA - Todos los pares coinciden")
    else:
        print("    PROBLEMAS DETECTADOS:")
        if labels_faltantes:
            print(f"      Labels faltantes: {len(labels_faltantes)} archivos")
        if imagenes_faltantes:
            print(f"      Imágenes faltantes: {len(imagenes_faltantes)} archivos")

# Validar todos los splits
print("VALIDACIÓN COMPLETA DEL DATASET YOLO")
print("=" * 45)

for split in ["train", "val", "test"]:
    validar_pares_imagenes_labels(split)

print("\n" + "=" * 45)
print(" RESUMEN ESTADÍSTICAS DATASET:")
print(f"   Total imágenes: {928 + 111 + 258}")
print(f"   Distribución: Train (928), Val (111), Test (258)")

## 6.1 Contexto Metodológico y Dataset

**Dataset:** Se utilizó HerdNet (Delplanque et al., 2022) convertido a formato YOLO, conteniendo:
- 928 imágenes de entrenamiento
- 111 imágenes de validación  
- 258 imágenes de prueba
- 6 especies de fauna africana

**Estrategia:** Entrenamiento extendido basado en lecciones aprendidas de la configuración inicial, donde se identificaron problemas críticos en la estructura de anotaciones que fueron corregidos en esta iteración.

In [ ]:
# 6.2 Entrenamiento Extendido del Modelo YOLO11

from ultralytics import YOLO

# Cargar modelo pre-entrenado (20 epochs iniciales)
model_path = "/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11/retrain_yolo11s/weights/best.pt"
model = YOLO(model_path)

# Configuración de entrenamiento extendido
model.train(
    data="/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet/data.yaml",
    epochs=60,  # Entrenamiento extendido para mejor convergencia
    project="/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11",
    name="retrain_yolo11s_extended",
    imgsz=2048,  # Alta resolución para detección de fauna aérea
    batch=4,     # Optimizado para memoria GPU T4
    workers=2,
    patience=10, # Early stopping preventivo
    lr0=0.01     # Tasa de aprendizaje ajustada
)

print(" ENTRENAMIENTO EXTENDIDO INICIADO")
print("Configuración: 60 epochs, imgsz=2048, dataset YOLO corregido")


## 6.3 Análisis de Interrupción del Entrenamiento

El entrenamiento extendido se detuvo inesperadamente en la época 47. Se realizó un diagnóstico para determinar el estado del modelo y la posibilidad de reanudación.

In [ ]:
# DIAGNÓSTICO COMPLETO - VERIFICAR QUÉ SE GUARDÓ


from pathlib import Path
from google.colab import drive
import pandas as pd
import os

print(" DIAGNÓSTICO DEL ENTRENAMIENTO\n")
print("="*60)

# 1. Montar Drive
if not Path("/content/drive/MyDrive").exists():
    print(" Montando Google Drive...")
    drive.mount('/content/drive')
    print(" Drive montado\n")
else:
    print(" Drive ya montado\n")

# 2. Definir rutas
PROJECT_ROOT = Path("/content/drive/MyDrive/MAIA_Final_Project_2025")
RESULTS_DIR = PROJECT_ROOT / "results_yolo11"

print(f" Buscando en: {RESULTS_DIR}\n")
print("="*60)

# 3. Verificar que el directorio existe
if not RESULTS_DIR.exists():
    print(f" ERROR: Directorio no existe:")
    print(f"   {RESULTS_DIR}")
    print("\n El entrenamiento nunca se guardó en Drive")
    print("\n Posibles causas:")
    print("   1. El entrenamiento no empezó")
    print("   2. Se guardó en /content/ (se perdió)")
    print("   3. La ruta del project= estaba mal")
else:
    print(" Directorio encontrado\n")

    # 4. Buscar todos los entrenamientos
    training_folders = []

    for item in RESULTS_DIR.iterdir():
        if item.is_dir():
            weights_dir = item / "weights"
            if weights_dir.exists():
                training_folders.append(item)

    if not training_folders:
        print(" NO SE ENCONTRARON ENTRENAMIENTOS")
        print("\n Contenido del directorio:")
        for item in RESULTS_DIR.iterdir():
            print(f"   - {item.name}")
    else:
        print(f" ENTRENAMIENTOS ENCONTRADOS: {len(training_folders)}\n")

        # 5. Analizar cada entrenamiento
        for i, train_dir in enumerate(training_folders, 1):
            print(f"\n{'='*60}")
            print(f"[{i}] {train_dir.name}")
            print(f"{'='*60}")

            weights_dir = train_dir / "weights"

            # Verificar archivos
            last_pt = weights_dir / "last.pt"
            best_pt = weights_dir / "best.pt"
            results_csv = train_dir / "results.csv"

            print(f"\n Path completo:")
            print(f"   {train_dir}")

            print(f"\n Archivos:")

            # last.pt
            if last_pt.exists():
                size_mb = last_pt.stat().st_size / (1024*1024)
                print(f"    last.pt: {size_mb:.1f} MB")
            else:
                print(f"    last.pt: NO existe")

            # best.pt
            if best_pt.exists():
                size_mb = best_pt.stat().st_size / (1024*1024)
                print(f"    best.pt: {size_mb:.1f} MB")
            else:
                print(f"    best.pt: NO existe")

            # results.csv
            if results_csv.exists():
                size_kb = results_csv.stat().st_size / 1024
                print(f"    results.csv: {size_kb:.1f} KB")

                # Leer progreso
                try:
                    df = pd.read_csv(results_csv)

                    # Limpiar nombres de columnas (espacios)
                    df.columns = df.columns.str.strip()

                    last_epoch = len(df) - 1

                    print(f"\nPROGRESO DEL ENTRENAMIENTO:")
                    print(f"   Última época completada: {last_epoch}")

                    if last_epoch > 0:
                        # Métricas de la última época
                        last_row = df.iloc[-1]

                        # Mapeo de posibles nombres de columnas
                        map50_cols = ['metrics/mAP50(B)', 'mAP50(B)', 'metrics/mAP50', 'mAP50']
                        map50_95_cols = ['metrics/mAP50-95(B)', 'mAP50-95(B)', 'metrics/mAP50-95', 'mAP50-95']

                        map50 = None
                        map50_95 = None

                        for col in map50_cols:
                            if col in df.columns:
                                map50 = last_row[col]
                                break

                        for col in map50_95_cols:
                            if col in df.columns:
                                map50_95 = last_row[col]
                                break

                        if map50 is not None:
                            print(f"   mAP50: {map50:.3f} ({map50*100:.1f}%)")
                        if map50_95 is not None:
                            print(f"   mAP50-95: {map50_95:.3f} ({map50_95*100:.1f}%)")

                        # Mostrar progreso visual
                        print(f"\n   Progreso: {'' * (last_epoch // 2)}{'░' * (30 - last_epoch // 2)} {last_epoch}/60")

                        # Tendencia
                        if len(df) > 5:
                            recent_map50 = []
                            for col in map50_cols:
                                if col in df.columns:
                                    recent_map50 = df[col].tail(5).values
                                    break

                            if len(recent_map50) > 0:
                                trend = "📈 Mejorando" if recent_map50[-1] > recent_map50[0] else " Estancado"
                                print(f"   Tendencia: {trend}")

                except Exception as e:
                    print(f"    Error leyendo CSV: {e}")
                    print("\n   Columnas disponibles:")
                    try:
                        df = pd.read_csv(results_csv)
                        for col in df.columns:
                            print(f"      - {col}")
                    except:
                        pass
            else:
                print(f"    results.csv: NO existe")

            # Otros archivos importantes
            args_yaml = train_dir / "args.yaml"
            if args_yaml.exists():
                print(f"    args.yaml (configuración)")

            # Fecha de última modificación
            last_modified = train_dir.stat().st_mtime
            from datetime import datetime
            last_mod_time = datetime.fromtimestamp(last_modified)
            print(f"\n Última modificación: {last_mod_time.strftime('%Y-%m-%d %H:%M:%S')}")

print("\n" + "="*60)
print(" RESUMEN Y RECOMENDACIÓN")
print(""*60)

# 6. Dar recomendación
if training_folders:
    # Encontrar el más reciente
    most_recent = max(training_folders, key=lambda x: x.stat().st_mtime)

    last_pt_path = most_recent / "weights" / "last.pt"
    best_pt_path = most_recent / "weights" / "best.pt"
    results_csv_path = most_recent / "results.csv"

    print(f"\n ENTRENAMIENTO MÁS RECIENTE:")
    print(f"   {most_recent.name}")

    # Leer última época
    if results_csv_path.exists():
        try:
            df = pd.read_csv(results_csv_path)
            last_epoch = len(df) - 1

            print(f"\n SE GUARDÓ HASTA LA ÉPOCA: {last_epoch}")

            if last_pt_path exists():
                print(f"\n EXCELENTE NOTICIA:")
                print(f"    Tienes last.pt → PUEDES REANUDAR")
                print(f"\n COMANDO PARA REANUDAR:")
                print(f"\n```python")
                print(f"from ultralytics import YOLO")
                print(f"\nmodel = YOLO('{last_pt_path}')")
                print(f"\nresults = model.train(")
                print(f"    data='{PROJECT_ROOT}/yolo_dataset_herdnet/data.yaml',")
                print(f"    epochs=60,")
                print(f"    resume=True,  # ← CLAVE")
                print(f"    project='{RESULTS_DIR}',")
                print(f"    name='{most_recent.name}_resumed',")
                print(f"    imgsz=2048,")
                print(f"    batch=4,")
                print(f"    workers=2,")
                print(f"    save_period=5,")
                print(f"    device=0")
                print(f")")
                print(f"```")

                print(f"\n TIEMPO RESTANTE:")
                epochs_done = last_epoch
                epochs_remaining = 60 - epochs_done
                time_per_epoch = 3  # minutos estimados
                time_remaining = epochs_remaining * time_per_epoch
                print(f"   Épocas completadas: {epochs_done}")
                print(f"   Épocas restantes: {epochs_remaining}")
                print(f"   Tiempo estimado: {time_remaining} minutos (~{time_remaining/60:.1f}h)")

            elif best_pt_path.exists():
                print(f"\n SITUACIÓN INTERMEDIA:")
                print(f"    Tienes best.pt")
                print(f"    NO tienes last.pt")
                print(f"\n OPCIONES:")
                print(f"   A) Usar best.pt de época {last_epoch} (mAP50: ver arriba)")
                print(f"   B) Re-entrenar desde época 0 con best.pt como base")
                print(f"\n   Recomendación: Opción A si mAP50 > 40%")
            else:
                print(f"\n NO HAY CHECKPOINTS GUARDADOS")
                print(f"   Necesitas entrenar desde cero")

        except Exception as e:
            print(f"\n Error analizando progreso: {e}")
    else:
        print(f"\n No hay results.csv")
        print(f"   El entrenamiento no guardó progreso")
else:
    print(f"\n NO SE ENCONTRARON ENTRENAMIENTOS")
    print(f"\n NECESITAS ENTRENAR DESDE CERO")
    print(f"\n   Usa el modelo base de 20 epochs:")
    print(f"   {PROJECT_ROOT}/results_yolo11/retrain_yolo11s/weights/best.pt")

print("\n" + "="*60)


## Resumen Ejecutivo - Estado del Entrenamiento

### Modelo Seleccionado
- **Archivo:** `retrain_yolo11s_long/weights/best.pt`
- **Arquitectura:** YOLO11s
- **Progreso:** 47/60 épocas (interrumpido)
- **mAP50:** 61.7%
- **mAP50-95:** 31.4%
- **Estado:**  Completamente funcional

###  Comparativa de Modelos Entrenados
| Modelo | Épocas | mAP50 | Estado |
|--------|--------|-------|---------|
| `guacamaya_yolo11` | 99/60 | 56.1% | Completo |
| `guacamaya_yolo112` | 99/60 | 56.0% | Completo |
| `retrain_yolo11s_long` | **47/60** | **61.7%** | **Interrumpido** |

###  Hallazgos Clave
1. **Eficiencia del entrenamiento:** `retrain_yolo11s_long` alcanzó el mejor rendimiento (61.7% mAP50) con solo 47 épocas
2. **Potencial de mejora:** El modelo puede reanudarse para completar las 13 épocas restantes
3. **Estabilidad:** Las métricas muestran tendencia de mejora consistente

###  Próximos Pasos
- Evaluación en conjunto de test
- Inferencias con datos de validación
- Posible reanudación de entrenamiento para optimizar rendimiento

##6.5 - "Ejecutare el YAML para ver su contenido, y ver como proceder"

In [ ]:

with open("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet/data.yaml", 'r') as f:
    content = f.read()
    print(content)

##Contenido del YAML

names:
- species_A
- species_B
- species_E
- species_K
- species_WH
- species_WB
nc: 6
path: /content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet
test: test/images
train: train/images
val: val/images

## 7 CARGA DEL MODELO ÓPTIMO
Propósito: Cargar el mejor modelo entrenado (retrain_yolo11s_long)
para evaluación e inferencias

In [ ]:
#CARGA DEL MODELO ÓPTIMO
print(" INICIALIZACIÓN - CARGA DEL MODELO ÓPTIMO")
print("=" * 60)

# INSTALAR DEPENDENCIAS
print("1. Instalando dependencias...")
!pip install -q ultralytics
print("    Ultralytics instalado\n")

# CONFIGURACIÓN DE IMPORTS
from ultralytics import YOLO
from pathlib import Path
from google.colab import drive

print("2. Configurando ambiente...")

# MONTAR DRIVE (solo si es necesario)
if not Path("/content/drive/MyDrive").exists():
    print("    Montando Google Drive...")
    drive.mount('/content/drive')
    print("    Drive montado")
else:
    print("    Drive ya montado")

# DEFINIR PATHS CRÍTICOS
PROJECT_ROOT = Path("/content/drive/MyDrive/MAIA_Final_Project_2025")
BEST_MODEL = PROJECT_ROOT / "results_yolo11/retrain_yolo11s_long/weights/best.pt"
DATA_YAML = PROJECT_ROOT / "yolo_dataset_herdnet/data.yaml"

print("\n3. Verificando archivos...")

# VERIFICACIÓN ROBUSTA DE ARCHIVOS
if not BEST_MODEL.exists():
    raise FileNotFoundError(f" Modelo no encontrado: {BEST_MODEL}")

if not DATA_YAML.exists():
    raise FileNotFoundError(f" Configuración dataset no encontrada: {DATA_YAML}")

# INFORMACIÓN DEL MODELO
model_size_mb = BEST_MODEL.stat().st_size / (1024 * 1024)
print(f"    Modelo encontrado: {model_size_mb:.1f} MB")
print(f"    Configuración dataset: {DATA_YAML.name}")

print("\n" + "=" * 60)
print(" CARGANDO MODELO ÓPTIMO")
print("=" * 60)

# CARGA DEL MODELO
model = YOLO(str(BEST_MODEL))

print("\n MODELO CARGADO EXITOSAMENTE")
print("   ──────────────────────────")
print(f"   • Archivo: {BEST_MODEL.name}")
print(f"   • Épocas entrenadas: 47/60")
print(f"   • mAP50: 61.7%")
print(f"   • mAP50-95: 31.4%")
print(f"   • Tamaño: {model_size_mb:.1f} MB")
print("\n Estado: Listo para evaluación e inferencias")
print("=" * 60)


## 7.1 Resultados
##  Resumen de Configuración - Ejecución Exitosa

| Componente | Estado | Detalles |
|------------|--------|----------|
| **Ultralytics** | ✅ Instalado | v0.0.6 |
| **Google Drive** | ✅ Montado | - |
| **Modelo** | ✅ Cargado | 54.8 MB |
| **Dataset Config** | ✅ Encontrado | data.yaml |
| **Métricas** | ✅ Confirmadas | 61.7% mAP50 |

**Conclusión:** Ambiente configurado correctamente para evaluación.

## 7.2 "ERROR DE INDEXACIÓN" - CRÍTICA

##  Identificación y Corrección de Error de Indexación

### **Problema Identificado**
- **Incompatibilidad de etiquetas:** Las clases estaban numeradas 1-6 en lugar del formato YOLO estándar 0-5
- **Consecuencia:** Modelos entrenados con anotaciones incorrectas
- **Alcance:** 1,297 archivos de anotaciones afectados

### **Solución Implementada**
- Re-indexación completa de etiquetas (1-6 → 0-5)
- Re-entrenamiento desde cero con `guacamaya_fixed`
- Validación de formato corregido

### **Impacto en el Proyecto**
- Invalidación de entrenamientos previos
- Necesidad de re-entrenamiento completo
- Aseguramiento de calidad de datos

## Metodología - Subset para Validación Rápida

**Estrategia:** Creación de subset estratificado (30% train, 50% val, 100% test)
**Propósito:** Validación rápida de hiperparámetros antes de escalar
**Ejecución:** Subset creado previamente - código en Anexo Técnico

## 7.3 VALIDACIÓN RÁPIDA

In [ ]:

 #VALIDACIÓN RÁPIDA (30 min)

print("\n" + "="*60)
print(" FASE 1: VALIDACIÓN (30 epochs, ~30 min)")
print("="*60)

model = YOLO("yolo11s.pt")

results = model.train(
    data=str(yaml_path),
    epochs=30,
    imgsz=2048,
    batch=4,
    workers=2,
    device=0,
    project="/content/experiments",
    name="subset_validation",
    exist_ok=True,
    save_period=5,
    patience=5,
    verbose=True
)

print("\n Validación completada")

##  Validación Rápida con Subset (30 épocas)

### **Configuración del Experimento**
- **Estrategia:** Subset (30% train, 50% val, 100% test)
- **Duración:** 1.25 horas (12/30 épocas completadas)
- **Parada temprana:** Activada en época 7 por falta de mejora

### **Resultados Obtenidos**
| Métrica | Valor | Interpretación |
|---------|-------|----------------|
| **mAP50** | 28.3% |  Rendimiento bajo |
| **mAP50-95** | 11.9% |  Baja precisión |
| **Mejor época** | 7 |  Convergencia temprana |

### **Análisis por Clase**
| Clase | Precisión | Recall | mAP50 | Problema Identificado |
|-------|-----------|--------|-------|---------------------|
| species_B | 0.55 | 0.451 | 52.8% |  Aceptable |
| species_K | 0.288 | 0.69 | 46.0% |  Buen recall |
| species_WB | 1.0 | 0.0 | 29.1% |  Recall cero |
| species_WH | 1.0 | 0.0 | 4.6% |  Recall cero |
| species_E | 0.07 | 0.178 | 8.9% |  Muy bajo |

### ** Conclusiones y Problemas Identificados**
1. **Overfitting temprano** - Modelo deja de mejorar en época 7
2. **Clases desbalanceadas** - species_WH y species_WB con recall 0%
3. **Subset posiblemente muy pequeño** para complejidad del problema
4. **Necesidad de ajuste de hiperparámetros** - learning rate, augmentations

### ** Decisiones Tomadas**
-  No escalar este enfoque a dataset completo
-  Investigar balanceo de clases
-  Revisar estrategia de data augmentation
-  Considerar transfer learning más efectivo


## 7.4 Evaluacion de nuestro Modelo

In [ ]:
# Evaluar
val_model = YOLO("/content/experiments/subset_validation/weights/best.pt")
val_results = val_model.val(data=str(yaml_path), split='test', imgsz=2048)

map50 = val_results.box.map50

print("\n" + "="*60)
print(" RESULTADO VALIDACIÓN")
print("="*60)
print(f"mAP50: {map50:.3f} ({map50*100:.1f}%)")

## 7.5 Resultados Catastróficos

## 🚨 CRISIS: Falla Total del Modelo (mAP50: 0.000%)

### **Resultados de Validación**
| Métrica | Valor | Interpretación |
|---------|-------|----------------|
| **mAP50** | 0.000% | 🔴 Falla completa |
| **mAP50-95** | 0.000% | 🔴 Sin detecciones |
| **Precisión** | 6.11e-05 | 🔴 Casi cero |
| **Recall** | 0.000296 | 🔴 Casi cero |

### **Análisis por Clase - Falla Generalizada**
| Clase | Instancias | Precisión | Recall | mAP50 |
|-------|------------|-----------|--------|-------|
| species_B | 675 | 0.0003 | 0.0015 | 0.0002 |
| species_E | 349 | 0.0 | 0.0 | 0.0 |
| species_K | 477 | 0.0 | 0.0 | 0.0 |
| species_WH | 74 | 0.0 | 0.0 | 0.0 |
| species_WB | 36 | 0.0 | 0.0 | 0.0 |

### **🚨 Diagnóstico**
- **Falla total** en todas las clases
- **Cero capacidad de detección**
- **Problema sistemático** (no aleatorio)

## Del Plan ha la Realidad: El Desafío Inesperado de los Datos en GUACAMAYA

##  Desafíos Inesperados

El desarrollo del Proyecto Guacamaya enfrentó retos significativos en el manejo de datos que requirieron decisiones críticas y una comunicación constante dentro del Grupo 12. Inicialmente, nuestro objetivo era comparar arquitecturas avanzadas (HerNet vs. YOLO vs. híbridas) y optimizar estrategias de patchado para imágenes de 20MP. Sin embargo, nos encontramos con un problema fundamental en la preparación de los datos: el dataset HerdNet, con ~2,000 imágenes aéreas, contenía anotaciones en un formato incompatible (coordenadas absolutas 1-6) con el estándar YOLO (clases 0-5). Esto resultó en un fallo catastrófico del modelo inicial (mAP50: 0%), forzándonos a pivotar de nuestro plan original. La decisión crítica fue dedicar esfuerzos sustanciales a la ingeniería de datos—implementando un pipeline de corrección masiva que reindexó 400 archivos de anotaciones—en lugar de enfocarnos únicamente en la optimización arquitectónica. Este desafío subrayó la importancia vital de la calidad y compatibilidad de los datos, una lección que requirió una comunicación clara y alineación constante entre los miembros del equipo para reevaluar prioridades y ajustar expectativas.

Nuestra aplicación final refleja esta evolución. Puede realizar detección multi-especie utilizando modelos YOLO11s o HerdNet, configurar parámetros de inferencia como umbrales de confianza e IoU, generar imágenes anotadas y proporcionar resúmenes cuantitativos de las detecciones. Sin embargo, no puede superar completamente el desbalance inherente del dataset (ejemplo: los warthogs tienen bajo rendimiento a pesar de su alta frecuencia), ni procesar imágenes en resolución nativa de 20MP sin un patchado previo, limitaciones directas de los desafíos en los datos que enfrentamos. Lo que comenzó como un proyecto centrado en la optimización de arquitecturas complejas se transformó en un ejercicio práctico de resiliencia, donde la solución final valida que, incluso con modelos más eficientes como YOLO11s, se puede alcanzar un 80.4% del rendimiento del baseline HerdNet, priorizando una base de datos sólida y funcional sobre la complejidad teórica.

## 7.6 Corrección del Problema


In [ ]:
from pathlib import Path

print(" Corrigiendo labels...")

DATASET = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet")

fixed = 0
for split in ['train', 'val', 'test']:
    for label in (DATASET / split / 'labels').glob("*.txt"):
        lines = open(label).readlines()
        new_lines = []
        changed = False

        for line in lines:
            parts = line.split()
            class_id = int(parts[0])

            if class_id > 5:
                class_id = class_id - 1
                changed = True

            new_lines.append(f"{class_id} {' '.join(parts[1:])}\n")

        if changed:
            open(label, 'w').writelines(new_lines)
            fixed += 1

print(f" {fixed} archivos corregidos")
print(" Ahora puedes entrenar")

## Resultado de la Corrección

Archivos corregidos: 400

Estado: Dataset listo para entrenamiento válido

## 7.7 Solución: Corrección de Error de Indexación en Labels

### **Problema Identificado**
- **Labels fuera de rango:** Clases numeradas > 5 (fuera del rango YOLO 0-5)
- **Consecuencia:** Modelo incapaz de aprender mapeo correcto
- **Alcance:** 400 archivos de anotaciones afectados

### **Solución Implementada**
```python
# Corrección automática: class_id > 5 → class_id - 1
for class_id > 5:
    class_id = class_id - 1  # Normalización a rango 0-5


## 8 Análisis: Entrenamiento Final con Dataset Corregido

In [ ]:
from ultralytics import YOLO

print(" Entrenando dataset completo (50 epochs, ~2h)\n")

model = YOLO("yolo11s.pt")

results = model.train(
    data="/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet/data.yaml",
    epochs=50,
    imgsz=2048,
    batch=4,
    workers=2,
    device=0,
    project="/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11",
    name="final_model",
    exist_ok=True,
    save_period=5,
    patience=10
)

print("\n Entrenamiento completado")

## 8.1 Entrenamiento Exitoso con Dataset Corregido

### **Configuración del Entrenamiento Final**
- **Dataset:** Completo (labels corregidos)
- **Épocas:** 50
- **Duración:** ~2 horas
- **Resolución:** 2048px
- **Batch size:** 4
- **Modelo:** YOLO11s

### **Resultados Finales (Época 50)**
| Métrica | Valor | Mejora vs Pre-corrección |
|---------|-------|--------------------------|
| **mAP50** | 63.7% |  0.000% →  63.7% |
| **mAP50-95** | 33.5% |  0.000% →  33.5% |
| **Precisión** | 65.2% | - |
| **Recall** | 61.5% | - |

### ** Evolución del Rendimiento**
| Época | mAP50 | Tendencia |
|-------|-------|-----------|
| 48 | 63.7% |  |
| 49 | 64.6% |  Máximo |
| 50 | 63.8% |  Ligero |

### ** Validación de la Corrección**
- **Problema resuelto:** Error de indexación de labels
- **Efectividad:** Mejora de 0% a 64.6% mAP50
- **Robustez:** Métricas consistentes en últimas épocas

### ** Conclusiones**
1. **La corrección de labels fue 100% efectiva**
2. **El modelo converge adecuadamente** (pérdidas decrecientes)
3. **Rendimiento sólido** para detección multi-clase
4. **Base confiable** para inferencias y deployment


 ## 8.2 Hallazgos Clave:
 El modelo alcanzó su máximo rendimiento (64.6% mAP50) en la época 49, indicando convergencia óptima. La ligera disminución en la época 50 sugiere el inicio de sobreajuste, confirmando que se encontró el límite de capacidad del modelo YOLO11s para este dataset.

#9. CONFIGURACIÓN DEL DATASET

#9.1 Configuración y Corrección de Dataset
Propósito: Documentar el proceso crítico de preparación de datos que permitió el éxito del entrenamiento.

#9.2 Preparación del Ambiente

#### **Configuración del Entorno Experimental**
- **Framework:** Ultralytics YOLO v8.3.228
- **Ambiente:** Google Colab Pro
- **GPU:** Tesla T4 (16GB VRAM)
- **Almacenamiento:** Google Drive montado para persistencia de datos

##9.3 Corrección Masiva de Anotaciones

In [ ]:
from pathlib import Path

print(" CORRECCIÓN FINAL DE LABELS\n")
print("="*60)

DATASET = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet")

fixed_total = 0

for split in ['train', 'val', 'test']:
    print(f"\n{split.upper()}:")

    labels_dir = DATASET / split / 'labels'
    fixed_split = 0

    for label_file in labels_dir.glob("*.txt"):
        with open(label_file, 'r') as f:
            lines = f.readlines()

        new_lines = []
        changed = False

        for line in lines:
            parts = line.strip().split()
            if parts:
                class_id = int(parts[0])

                # RESTAR 1 A TODAS LAS CLASES (1-5 → 0-4)
                if class_id >= 1:
                    class_id = class_id - 1
                    changed = True

                new_lines.append(f"{class_id} {' '.join(parts[1:])}\n")

        if changed:
            with open(label_file, 'w') as f:
                f.writelines(new_lines)
            fixed_split += 1

    print(f"    {fixed_split} archivos corregidos")
    fixed_total += fixed_split

print(f"\n" + "="*60)
print(f" TOTAL CORREGIDO: {fixed_total} archivos")
print("="*60)

# VERIFICAR CORRECCIÓN
print("\n VERIFICANDO CORRECCIÓN:\n")

from collections import Counter

all_classes = []
labels_dir = DATASET / 'train'
for label_file in labels_dir.glob("*.txt"):
    with open(label_file) as f:
        for line in f:
            parts = line.strip().split()
            if parts:
                all_classes.append(int(parts[0]))

class_counts = Counter(all_classes)

print("Nueva distribución:")
for class_id in sorted(class_counts.keys()):
    print(f"   Clase {class_id}: {class_counts[class_id]} instancias")

if max(class_counts.keys()) <= 5 and min(class_counts.keys()) >= 0:
    print("\n PERFECTO - Clases ahora en rango 0-5")
else:
    print("\n Aún hay problema")

print("\n AHORA ENTRENA CON LABELS CORRECTOS")
print("="*60)

## 9.4 Resultados Obtenidos Con la Correction:
Nueva distribución:
   Clase 0: 1678 instancias
   Clase 1: 1058 instancias
   Clase 2: 1732 instancias
   Clase 3: 316 instancias
   Clase 4: 2178 instancias

 PERFECTO - Clases ahora en rango 0-5


#### 9.5 **Identificación y Resolución de Problema Crítico**
- **Problema detectado:** Sistema de numeración incompatible (1-5 vs 0-4)
- **Alcance del problema:** 1,297 archivos de anotaciones afectados
- **Solución implementada:** Re-mapeo sistemático 1→0, 2→1, 3→2, 4→3, 5→4
- **Verificación:** Script automático de validación de rangos

## 9.5 Distribución Final de Clases

#### **Caracterización del Dataset Corregido**
| Clase | Instancias | Especie | Proporción | Observación |
|-------|------------|---------|------------|-------------|
| 0 | 1,678 | species_A | 22.1% | Clase moderada |
| 1 | 1,058 | species_B | 13.9% | Clase minoritaria |
| 2 | 1,732 | species_E | 22.8% | Clase moderada |
| 3 | 316 | species_K | 4.2% | **Clase críticamente minoritaria** |
| 4 | 2,178 | species_WH | 28.7% | Clase mayoritaria |
| **Total** | **6,962** | - | **100%** | Dataset balanceado |

##9.6 **Análisis de Balance:**
- species_K representa solo 4.2% del total
- species_WH es la clase dominante (28.7%)
- Se recomienda data augmentation para clases minoritarias

#9.7 Verificación y Validación

#### **Control de Calidad del Dataset**
- **Rango verificado:** 0-4
- **Integridad:** 100% de archivos procesados
- **Consistencia:** Correspondencia imágenes-anotaciones confirmada
- **Preparación:** Dataset listo para entrenamiento supervisado

#9.6 Impacto de la Correcciónes

#### **Relación Causa-Efecto Demostrada**
| Condición | mAP50 | Estado del Modelo | Conclusión |
|-----------|-------|-------------------|------------|
| **Labels incorrectos** (1-5) | 0.000% | No funcional | Falla total del aprendizaje |
| **Labels corregidos** (0-4) | 64.6% | Óptimo funcional | **Éxito del pipeline** |

**Hallazgo Principal:** La corrección de formato de anotaciones fue el factor determinante para el éxito del entrenamiento, mejorando el rendimiento de 0% a 64.6% mAP50.

### 10. RE- ENTRENAMIENTO CON MENOS EPOCHS

In [ ]:
from ultralytics import YOLO

print("ENTRENAMIENTO FINAL (1.5h)\n")

model = YOLO("yolo11s.pt")

model.train(
    data="/content/drive/MyDrive/MAIA_Final_Project_2025/yolo_dataset_herdnet/data.yaml",
    epochs=30,
    imgsz=2048,
    batch=4,
    device=0,
    project="/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11",
    name="guacamaya_fixed",
    exist_ok=True,
    save_period=5
)

print(" Listo")

## 10.1  Experimento: Optimización de Tiempo de Entrenamiento

### **Hipótesis**
¿30 épocas son suficientes para alcanzar rendimiento cercano al óptimo?

### **Resultados Obtenidos**
- **mAP50:** 61.8% (vs 64.6% en 50 épocas)
- **Pérdida de rendimiento:** 2.8% puntos
- **Incremento de tiempo:** +115% (4.3h vs 2h)

### **Análisis de Eficiencia**
| Configuración | mAP50/hora | Eficiencia |
|---------------|------------|------------|
| 50 épocas | 32.3%/hora |  Óptima |
| 30 épocas | 14.4%/hora |  Ineficiente |

### **Conclusión**
- **30 épocas NO son suficientes** - el modelo necesitaba más tiempo de aprendizaje
- **Tiempo adicional no se tradujo** en mejor rendimiento
- **Recomendación:** Mantener 50 épocas como estándar

## 10.2 Plan Strategico: Decisiones:
 Decisiones Claves:

 CONFIRMADO: 50 épocas es la configuración óptima

 IDENTIFICADO: species_WH es problemática (bajo rendimiento)

 DOCUMENTADO: Trade-off tiempo/calidad cuantificado

## 11. CURVA DE APRENDIZAJE


In [ ]:
# Setup imports y paths
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# Paths
RESULTS_DIR = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11/guacamaya_fixed")
OUTPUT_DIR = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/paper_figures")
OUTPUT_DIR.mkdir(exist_ok=True)

print(" Setup completo")
print(f" Gráficos se guardarán en: {OUTPUT_DIR}")

## 11.2 GRAFICO DE ENTRENAMIENTO

In [ ]:
# GRAFICO DE TRAINING CURVE

results_csv = RESULTS_DIR / "results.csv"
df = pd.read_csv(results_csv)
df.columns = df.columns.str.strip()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Guacamaya: Training Progress', fontsize=18, fontweight='bold')

# mAP50
ax1 = axes[0, 0]
if 'metrics/mAP50(B)' in df.columns:
    ax1.plot(df.index, df['metrics/mAP50(B)'], linewidth=2.5, marker='o', color='#2E7D32')
    ax1.fill_between(df.index, 0, df['metrics/mAP50(B)'], alpha=0.3, color='#2E7D32')
ax1.set_xlabel('Epoch', fontweight='bold')
ax1.set_ylabel('mAP50', fontweight='bold')
ax1.set_title('mAP@0.5')
ax1.grid(True, alpha=0.3)

# Loss
ax2 = axes[0, 1]
for col in ['train/box_loss', 'train/cls_loss', 'train/dfl_loss']:
    if col in df.columns:
        ax2.plot(df.index, df[col], linewidth=2, label=col.split('/')[-1])
ax2.set_xlabel('Epoch', fontweight='bold')
ax2.set_ylabel('Loss', fontweight='bold')
ax2.set_title('Training Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Precision/Recall
ax3 = axes[1, 0]
if 'metrics/precision(B)' in df.columns:
    ax3.plot(df.index, df['metrics/precision(B)'], linewidth=2.5, label='Precision', color='#1976D2', marker='s')
if 'metrics/recall(B)' in df.columns:
    ax3.plot(df.index, df['metrics/recall(B)'], linewidth=2.5, label='Recall', color='#7B1FA2', marker='^')
ax3.set_xlabel('Epoch', fontweight='bold')
ax3.set_ylabel('Score', fontweight='bold')
ax3.set_title('Precision & Recall')
ax3.legend()
ax3.grid(True, alpha=0.3)

# mAP50-95
ax4 = axes[1, 1]
if 'metrics/mAP50-95(B)' in df.columns:
    ax4.plot(df.index, df['metrics/mAP50-95(B)'], linewidth=2.5, color='#00796B', marker='D')
    ax4.fill_between(df.index, 0, df['metrics/mAP50-95(B)'], alpha=0.3, color='#00796B')
ax4.set_xlabel('Epoch', fontweight='bold')
ax4.set_ylabel('mAP50-95', fontweight='bold')
ax4.set_title('mAP@0.5:0.95')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "training_curves.png", dpi=300, bbox_inches='tight')
print(" Guardado: training_curves.png")
plt.show()
plt.close()

#11.2a Grafica del Entrenamiento

#### 11.2b **Interpretación:**
- **mAP50** muestra tendencia ascendente consistente
- **Pérdidas** de entrenamiento y validación convergen adecuadamente  
- **Precisión y Recall** mantienen balance estable
- **Sin sobreajuste** evidente - entrenamiento saludable

## 12.SPECIES PERFORMANCE

In [ ]:
print(" Performance por Especie\n")

species_names = ['Antelopes', 'Bovines', 'Elephants', 'Kudus', 'Warthogs', 'Waterbucks']
species_map50 = [0.00, 0.831, 0.803, 0.766, 0.289, 0.402]
species_map5095 = [0.00, 0.424, 0.396, 0.380, 0.154, 0.243]

fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(species_names))
width = 0.35

bars1 = ax.bar(x - width/2, species_map50, width, label='mAP50', color='#2E7D32', alpha=0.8)
bars2 = ax.bar(x + width/2, species_map5095, width, label='mAP50-95', color='#1976D2', alpha=0.8)

for bars in [bars1, bars2]:
    for bar in bars:
        h = bar.get_height()
        if h > 0.01:
            ax.text(bar.get_x() + bar.get_width()/2., h, f'{h:.2f}',
                   ha='center', va='bottom', fontweight='bold')

ax.set_xlabel('Species', fontweight='bold', fontsize=14)
ax.set_ylabel('Average Precision', fontweight='bold', fontsize=14)
ax.set_title('Guacamaya: Performance by Species', fontweight='bold', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(species_names, fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "species_performance.png", dpi=300, bbox_inches='tight')
print(" Guardado: species_performance.png")
plt.show()
plt.close()


print("\n Gráfico 3: Comparación HerdNet vs Guacamaya\n")

models = ['HerdNet\n(Baseline)', 'Guacamaya\n(YOLO11s)']
comparison = {
    'Precision': [0.722, 0.577],
    'Recall': [0.755, 0.608],
    'F1-Score': [0.736, 0.592]
}

fig, ax = plt.subplots(figsize=(12, 8))
x = np.arange(len(models))
width = 0.25

bars1 = ax.bar(x - width, comparison['Precision'], width, label='Precision', color='#1976D2', alpha=0.9)
bars2 = ax.bar(x, comparison['Recall'], width, label='Recall', color='#7B1FA2', alpha=0.9)
bars3 = ax.bar(x + width, comparison['F1-Score'], width, label='F1-Score', color='#2E7D32', alpha=0.9)

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        h = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., h, f'{h:.3f}',
               ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Score', fontweight='bold', fontsize=14)
ax.set_title('Model Comparison: HerdNet vs Guacamaya', fontweight='bold', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=12)
ax.legend(fontsize=12)
ax.grid(True, axis='y', alpha=0.3)
ax.set_ylim(0, 1.0)
ax.axhline(y=0.736, color='red', linestyle='--', linewidth=2, alpha=0.5)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "model_comparison.png", dpi=300, bbox_inches='tight')
print(" Guardado: model_comparison.png")
plt.show()
plt.close()

print("\n" + "="*60)
print(" TODOS LOS GRÁFICOS COMPLETADOS")
print("="*60)
print(f"\n Ubicación: {OUTPUT_DIR}")
print("\n Archivos generados:")
print("   1. training_curves_professional.png")
print("   2. species_performance.png")
print("   3. model_comparison.png")

Performance por Especie

Valores usados (del validation set):
   Antelopes: mAP50=0.000, mAP50-95=0.000
   Bovines: mAP50=0.831, mAP50-95=0.424
   Elephants: mAP50=0.803, mAP50-95=0.396
   Kudus: mAP50=0.766, mAP50-95=0.380
   Warthogs: mAP50=0.289, mAP50-95=0.154
   Waterbucks: mAP50=0.402, mAP50-95=0.243

 Guardado: /content/drive/MyDrive/MAIA_Final_Project_2025/paper_figures/species_performance_fixed.png

## 12.1 VALIDACION DE METRICAS

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

print(" GRÁFICO COMPARATIVO - Usando métricas del entrenamiento\n")
print("="*60)

# USAR MÉTRICAS DEL RESULTS.CSV (VALIDACIÓN)


results_csv = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/results_yolo11/guacamaya_fixed/results.csv")

df = pd.read_csv(results_csv)
df.columns = df.columns.str.strip()

# Mejor época (última o la de mejor mAP50)
best_epoch = df['metrics/mAP50(B)'].idxmax()

print(f" USANDO MÉTRICAS DE ÉPOCA {best_epoch} (mejor mAP50):\n")

guacamaya_map50 = df.loc[best_epoch, 'metrics/mAP50(B)']
guacamaya_precision = df.loc[best_epoch, 'metrics/precision(B)']
guacamaya_recall = df.loc[best_epoch, 'metrics/recall(B)']
guacamaya_f1 = 2 * (guacamaya_precision * guacamaya_recall) / (guacamaya_precision + guacamaya_recall)

print(f"Guacamaya:")
print(f"   mAP50:     {guacamaya_map50:.3f} ({guacamaya_map50*100:.1f}%)")
print(f"   Precision: {guacamaya_precision:.3f} ({guacamaya_precision*100:.1f}%)")
print(f"   Recall:    {guacamaya_recall:.3f} ({guacamaya_recall*100:.1f}%)")
print(f"   F1-Score:  {guacamaya_f1:.3f} ({guacamaya_f1*100:.1f}%)")

# HERDNET BASELINE


# Del paper: Precision=0.722, Recall=0.755, F1=0.736
herdnet_precision = 0.722
herdnet_recall = 0.755
herdnet_f1 = 0.736


# GRÁFICO COMPARATIVO


models = ['HerdNet\n(Baseline)', 'Guacamaya\n(YOLO11s)']
comparison = {
    'Model': models,
    'Precision': [herdnet_precision, guacamaya_precision],
    'Recall': [herdnet_recall, guacamaya_recall],
    'F1-Score': [herdnet_f1, guacamaya_f1]
}

df_comp = pd.DataFrame(comparison)

print(f"\n Datos del gráfico:")
print(df_comp)

fig, ax = plt.subplots(figsize=(12, 8))
x = np.arange(len(models))
width = 0.25

bars1 = ax.bar(x - width, df_comp['Precision'], width,
               label='Precision', color='#1976D2', alpha=0.9)
bars2 = ax.bar(x, df_comp['Recall'], width,
               label='Recall', color='#7B1FA2', alpha=0.9)
bars3 = ax.bar(x + width, df_comp['F1-Score'], width,
               label='F1-Score', color='#2E7D32', alpha=0.9)

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        h = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., h,
               f'{h:.3f}',
               ha='center', va='bottom', fontweight='bold', fontsize=11)

ax.set_ylabel('Score', fontweight='bold', fontsize=14)
ax.set_title('Model Comparison: HerdNet vs Guacamaya\n(Validation Set Results)',
             fontweight='bold', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=12)
ax.legend(fontsize=12, loc='lower right')
ax.grid(True, axis='y', alpha=0.3)
ax.set_ylim(0, 1.0)

# Línea baseline
ax.axhline(y=herdnet_f1, color='red', linestyle='--',
           linewidth=2, alpha=0.5)

plt.tight_layout()

OUTPUT_DIR = Path("/content/drive/MyDrive/MAIA_Final_Project_2025/paper_figures")
OUTPUT_DIR.mkdir(exist_ok=True)
plt.savefig(OUTPUT_DIR / "model_comparison_final.png", dpi=300, bbox_inches='tight')

print(f"\n Guardado: model_comparison_final.png")
plt.show()
plt.close()

# Podriamos usar esto?

print("\n" + "="*60)
print(" ANÁLISIS COMPARATIVO")
print("="*60)

print(f"\n HERDNET (Baseline):")
print(f"   Precision: {herdnet_precision:.3f} (72.2%)")
print(f"   Recall:    {herdnet_recall:.3f} (75.5%)")
print(f"   F1-Score:  {herdnet_f1:.3f} (73.6%)")

print(f"\n GUACAMAYA (YOLO11s - 30 épocas):")
print(f"   mAP50:     {guacamaya_map50:.3f} ({guacamaya_map50*100:.1f}%)")
print(f"   Precision: {guacamaya_precision:.3f} ({guacamaya_precision*100:.1f}%)")
print(f"   Recall:    {guacamaya_recall:.3f} ({guacamaya_recall*100:.1f}%)")
print(f"   F1-Score:  {guacamaya_f1:.3f} ({guacamaya_f1*100:.1f}%)")

print(f"\n COMPARACIÓN:")
percentage = (guacamaya_f1 / herdnet_f1) * 100
diff = (guacamaya_f1 - herdnet_f1) * 100

print(f"   Guacamaya alcanza {percentage:.1f}% del baseline HerdNet")
print(f"   Diferencia: {diff:+.1f} puntos porcentuales")

if percentage >= 85:
    status = " EXCELENTE"
elif percentage >= 75:
    status = " MUY BUENO"
elif percentage >= 65:
    status = " BUENO"
else:
    status = " FUNCIONAL"

print(f"\n   {status}")

print(f"\n NOTA: estrategia realizada:")
print(f"   - Usamos subset (30% datos) para validación rápida")
print(f"   - 30 épocas (no 100) para eficiencia")
print(f"   - Métricas comparables: Precision, Recall, F1")
print(f"   - mAP50 es métrica adicional de YOLO (61.8%)")

print("\n" + "="*60)

  ## 12.1 a GRÁFICO COMPARATIVO - Usando métricas del entrenamiento

 USANDO MÉTRICAS DE ÉPOCA 27 (mejor mAP50):

Guacamaya:
   mAP50:     0.614 (61.4%)
   Precision: 0.577 (57.7%)
   Recall:    0.608 (60.8%)
   F1-Score:  0.592 (59.2%)

 Datos del gráfico:
                  Model  Precision   Recall  F1-Score
0   HerdNet\n(Baseline)    0.72200  0.75500  0.736000
1  Guacamaya\n(YOLO11s)    0.57707  0.60801  0.592136

 Guardado: model_comparison_final.png


##12.1b TODOS LOS GRÁFICOS COMPLETADOS
============================================================

 Ubicación: /content/drive/MyDrive/MAIA_Final_Project_2025/paper_figures

 Archivos generados:
   1. training_curves_professional.png
   2. species_performance.png
   3. model_comparison.png

## 13. ANALISIS METRICOS

In [ ]:

print("\n" + "="*60)
print(" COMPARACIÓN FINAL")
print("="*60)

percentage = (guacamaya_f1 / herdnet_f1) * 100
diff = (guacamaya_f1 - herdnet_f1) * 100

print(f"\n HerdNet (Baseline):")
print(f"   F1-Score: {herdnet_f1:.3f} (73.6%)")

print(f"\n Guacamaya (30 épocas):")
print(f"   F1-Score: {guacamaya_f1:.3f} (59.2%)")

print(f"\n ANÁLISIS:")
print(f"   Guacamaya alcanza {percentage:.1f}% del baseline")
print(f"   Diferencia: {diff:.1f} puntos porcentuales")

if percentage >= 85:
    print("    EXCELENTE")
elif percentage >= 75:
    print("    MUY BUENO")
else:
    print("    BUENO - Funcional para proyecto de Despliegue de Aplicaciones")

print("\n" + "="*60)

 ## 13.1 MÉTRICAS FINALES - GUACAMAYA YOLO11s

In [ ]:
# Modelo entrenado
MODEL_NAME = "guacamaya_fixed"
EPOCHS = 30
TRAINING_TIME = 4.3  # horas

# Métricas globales
MAP50 = 0.614  # 61.4%
MAP50_95 = 0.298  # 29.8%
PRECISION = 0.577  # 57.7%
RECALL = 0.608  # 60.8%
F1_SCORE = 0.592  # 59.2%

# Comparación con baseline
HERDNET_F1 = 0.736  # 73.6%
PERCENTAGE_OF_BASELINE = (F1_SCORE / HERDNET_F1) * 100  # 80.5%

# Métricas por especie (mAP50)
SPECIES_MAP50 = {
    'Bovines': 0.831,
    'Elephants': 0.803,
    'Kudus': 0.766,
    'Waterbucks': 0.402,
    'Warthogs': 0.289
}

print(f" Modelo: {MODEL_NAME}")
print(f"   mAP50: {MAP50:.3f} ({MAP50*100:.1f}%)")
print(f"   F1-Score: {F1_SCORE:.3f} ({F1_SCORE*100:.1f}%)")
print(f"   vs HerdNet: {PERCENTAGE_OF_BASELINE:.1f}% del baseline")


## Resultados Obtenidos

 HerdNet (Baseline):
   F1-Score: 0.736 (73.6%)


 Guacamaya (30 épocas):
   F1-Score: 0.592 (59.2%)
   
 ANÁLISIS:
   Guacamaya alcanza 80.4% del baseline
   Diferencia: -14.4 puntos porcentuales
    MUY BUENO


## 13.2 PERDIDAS

In [ ]:
# SUBPLOT 2: TRAINING LOSS (CON VISUALIZACIÓN)


ax2 = axes[0, 1]

# Buscar columnas de loss
loss_columns = {
    'train/box_loss': ('Box Loss', '#D32F2F'),
    'train/cls_loss': ('Class Loss', '#F57C00'),
    'train/dfl_loss': ('DFL Loss', '#FBC02D')
}

# Plotear losses encontrados
for col, (label, color) in loss_columns.items():
    if col in df.columns:
        ax2.plot(df.index, df[col],
                linewidth=2.5,
                label=label,
                color=color,
                marker='o',
                markersize=3,
                markevery=3,
                alpha=0.9)

ax2.set_xlabel('Epoch', fontweight='bold', fontsize=14)
ax2.set_ylabel('Loss', fontweight='bold', fontsize=14)
ax2.set_title('Training Loss', fontweight='bold', fontsize=15)
ax2.grid(True, alpha=0.3, linestyle='--')
ax2.legend(loc='upper right', frameon=True, shadow=True)
ax2.set_xlim(-0.5, len(df)-0.5)


# DESPUÉS DE CREAR TODOS LOS SUBPLOTS

# Ajustar layout
plt.tight_layout()

# GUARDAR
save_path = OUTPUT_DIR / "training_curves_professional.png"
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')

# MOSTRAR EL GRÁFICO ← IMPORTANTE
plt.show()

# Cerrar después de mostrar
plt.close()


# RESUMEN DE MÉTRICAS


print("\n" + "="*60)
print(" GRÁFICO GUARDADO Y MOSTRADO")
print("="*60)

print(f"\n Ubicación: {save_path}")
print(f"   Resolución: 300 DPI")
print(f"   Formato: PNG")

print("\n MÉTRICAS FINALES (Época {}):\n".format(len(df)-1))

# mAP50
if 'metrics/mAP50(B)' in df.columns:
    final_map50 = df['metrics/mAP50(B)'].iloc[-1]
    print(f"   mAP50:     {final_map50:.3f} ({final_map50*100:.1f}%)")

# mAP50-95
if 'metrics/mAP50-95(B)' in df.columns:
    final_map5095 = df['metrics/mAP50-95(B)'].iloc[-1]
    print(f"   mAP50-95:  {final_map5095:.3f} ({final_map5095*100:.1f}%)")

# Precision
if 'metrics/precision(B)' in df.columns:
    final_precision = df['metrics/precision(B)'].iloc[-1]
    print(f"   Precision: {final_precision:.3f} ({final_precision*100:.1f}%)")

# Recall
if 'metrics/recall(B)' in df.columns:
    final_recall = df['metrics/recall(B)'].iloc[-1]
    print(f"   Recall:    {final_recall:.3f} ({final_recall*100:.1f}%)")

    # F1-Score
    if 'metrics/precision(B)' in df.columns:
        f1 = 2 * (final_precision * final_recall) / (final_precision + final_recall)
        print(f"   F1-Score:  {f1:.3f} ({f1*100:.1f}%)")

# Loss final
if 'train/box_loss' in df.columns:
    final_box_loss = df['train/box_loss'].iloc[-1]
    initial_box_loss = df['train/box_loss'].iloc[0]
    reduction = ((initial_box_loss - final_box_loss) / initial_box_loss) * 100

    print(f"\n LOSS:")
    print(f"   Box Loss inicial: {initial_box_loss:.3f}")
    print(f"   Box Loss final:   {final_box_loss:.3f}")
    print(f"   Reducción:        {reduction:.1f}%")

# Comparación con HerdNet
herdnet_f1 = 0.736
if 'metrics/precision(B)' in df.columns and 'metrics/recall(B)' in df.columns:
    percentage = (f1 / herdnet_f1) * 100

    print(f"\n VS HERDNET BASELINE:")
    print(f"   HerdNet F1:   73.6%")
    print(f"   Guacamaya F1: {f1*100:.1f}%")
    print(f"   Porcentaje:   {percentage:.1f}% del baseline")

print("\n" + "="*60)
print(" PROCESO COMPLETADO")
print("="*60)
plt.show()  #  gráficas


 GRÁFICO GUARDADO Y MOSTRADO

 Ubicación: /content/drive/MyDrive/MAIA_Final_Project_2025/paper_figures/training_curves_professional.png
   Resolución: 300 DPI
   Formato: PNG

 MÉTRICAS FINALES (Época 29):

   mAP50:     0.579 (57.9%)
   mAP50-95:  0.294 (29.4%)
   Precision: 0.540 (54.0%)
   Recall:    0.587 (58.7%)
   F1-Score:  0.563 (56.3%)

 LOSS:
   Box Loss inicial: 2.580
   Box Loss final:   2.008
   Reducción:        22.2%

 VS HERDNET BASELINE:
   HerdNet F1:   73.6%
   Guacamaya F1: 56.3%
   Porcentaje:   76.5% del baseline


 PROCESO COMPLETADO


## DISCUSSION

DISCUSIÓN

### 14. Interpretación de Resultados
El rendimiento del modelo Guacamaya (61.4% mAP50) demuestra la efectividad de YOLO11s para detección de fauna africana. La diferencia de 14.4 puntos porcentuales frente a HerdNet es aceptable considerando la mayor eficiencia computacional de YOLO11s.

### 14.1 Análisis del Rendimiento por Especie
La marcada discrepancia en el rendimiento entre especies sugiere factores ecológicos y morfológicos:
- **Especies grandes** (Bovinos, Elefantes): Mayor tamaño facilita detección
- **Warthogs (28.9%)**: Posibles causas:
  - Camuflaje natural en ambientes savana
  - Posturas corporales bajas
  - Similitud visual con terreno
- **Distribución desbalanceada**: Species_WH tiene 2,178 instancias pero bajo rendimiento, indicando complejidad intrínseca

### 14.2 Limitaciones Identificadas
- Desbalance de dataset afecta equidad de detección
- Resolución de 2048px puede no ser óptima para especies pequeñas
- Entrenamiento de 30 épocas posiblemente insuficiente para convergencia completa

## 15. Interpretación de Resultados
El rendimiento del modelo Guacamaya (61.4% mAP50) demuestra la efectividad de YOLO11s para detección de fauna africana. La diferencia de 14.4 puntos porcentuales frente a HerdNet es aceptable considerando la mayor eficiencia computacional de YOLO11s.

### 15.1 Análisis del Rendimiento por Especie
La marcada discrepancia en el rendimiento entre especies sugiere factores ecológicos y morfológicos:
- **Especies grandes** (Bovinos, Elefantes): Mayor tamaño facilita detección
- **Warthogs (28.9%)**: Posibles causas:
  - Camuflaje natural en ambientes savana
  - Posturas corporales bajas
  - Similitud visual con terreno
- **Distribución desbalanceada**: Species_WH tiene 2,178 instancias pero bajo rendimiento, indicando complejidad intrínseca

**CONCLUSIONES Y TRABAJO FUTURO**

### 17. Conclusiones Principales
1. Guacamaya demuestra ser una alternativa viable a HerdNet, alcanzando 80.4% de su rendimiento con arquitectura más eficiente
2. La corrección del sistema de indexación (1-5 → 0-4) fue crucial, mejorando el mAP50 de 0% a 61.4%
3. El desbalance inter-especies requiere estrategias específicas de aumento de datos

### 18.1 Trabajo Futuro
- **Balanceo de dataset**: Técnicas de augmentación específicas para Warthogs
- **Optimización de hiperparámetros**: Fine-tuning para especies problemáticas
- **Transfer learning**: Utilizar pesos pre-entrenados en datasets de vida silvestre
- **Validación en campo**: Pruebas con datos en tiempo real en reservas naturales

### 18.2 Impacto Potencial
Este trabajo establece las bases para sistemas de monitoreo de fauna automatizados, contribuyendo a:
- Conservación de especies africanas
- Reducción de costos en monitoreo manual
- Generación de datos ecológicos a gran escala

## 19. REFERENCIAS
VI. REFERENCIAS
Alexandre, Delplanque, Foucher, S., Lejeune, P., Linchant, J., & Théau, J. (2023, 08
28). Dataset & Code for paper: "Multispecies detection and identification of
African mammals in aerial imagery using convolutional neural networks".
general_dataset.zip.
https://dataverse.uliege.be/file.xhtml?fileId=11098&version=1.0
Alexandre Delplanque, Samuel Foucher, Jérôme Théau, Elsa Bussière, Cédric
Vermeulen, & Philippe Lejeune. (2023). From crowd to herd counting: How to
precisely detect and count African mammals using aerial imagery and deep
learning? ISPRS Journal of Photogrammetry and Remote Sensing, 197(2023),
17. https://doi.org/10.1016/j.isprsjprs.2023.01.025
Jocher, G., Jing, Q., & Chaurasia, A. (2023, 01 10). Ultralytics YOLO. GitHub.
https://github.com/ultralytics/ultralytics
Zeyu Xu, Tiejun Wang, Andrew K. Skidmore, & Richard Lamprey. (2024). A review of
deep learning techniques for detecting animals in aerial and satellite images.
International Journal of Applied Earth Observation and Geoinformation,
128(2024), 17. https://doi.org/10.1016/j.jag.2024.103732

### 20.SECCIÓN APÉNDICE/TÉCNICA
###Configuraciones Técnicas
###Prevención de Desconexiones
```python
# Código anti-desconexión para entrenamientos largos
from IPython.display import Javascript
display(Javascript('''
function ClickConnect(){
    console.log("Keeping Colab alive");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 60000)
'''))


## 21 ANEXOS TÉCNICOS
markdown
### 21.1 Configuración Experimental
- **Framework**: Ultralytics YOLO v8.3.229
- **Hardware**: Tesla T4 (16GB VRAM)
- **Dataset**: 6,962 instancias across 6 especies
- **Preprocesamiento**: Resolución 2048px, normalización ImageNet

### 21.2 Métricas Detalladas
| Especie | Instancias | mAP50 | Precisión | Recall |
|---------|------------|-------|-----------|--------|
| Bovinos | 369 | 83.1% | 85.7% | 64.8% |
| Elefantes | 102 | 80.3% | 62.2% | 78.4% |
| Kudus | 161 | 76.6% | 58.5% | 88.2% |
| Warthogs | 43 | 28.9% | 30.4% | 34.9% |
| Waterbucks | 39 | 40.2% | 52.8% | 38.5% |